
<img src="resources/cropped-SummerWorkshop_Header.png">  

<h1 align="center">EM Connectomics - Question 1</h1> 
<h2 align="center">Summer Workshop on the Dynamic Brain </h2> 
<h3 align="center">XXX, August XX, XXXX</h3> 

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">


</div>



<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
    <p><b>Task 1:</b> This question explores some of basic functionality needed for investigating the EM dataset.
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 1.1:</b> Import the key modules and set parameters

</div>

In [36]:
from caveclient import CAVEclient
from meshparty import trimesh_io, trimesh_vtk
import pandas as pd
import numpy as np
client = CAVEclient('minnie65_public_v117')


In [33]:
syn_table_resolution = client.materialize.get_table_metadata(client.materialize.synapse_table)['voxel_resolution']

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
    <p><b> Task 1.2:</b> From the database, find all the 'cleaned' axons that are predicted to be inhibitory cells. 
      </div>

In [3]:
client.materialize.get_tables()

['nucleus_detection_v0',
 'synapses_pni_2',
 'nucleus_neuron_svm',
 'proofreading_status_public_release',
 'func_unit_em_match_release',
 'allen_soma_ei_class_model_v1',
 'allen_visp_column_soma_coarse_types_v1']

In [17]:
cln_df=client.materialize.query_table('proofreading_status_public_release', 
                              filter_equal_dict={'status_axon': 'clean'})
ei_df = client.materialize.query_table('allen_soma_ei_class_model_v1')


In [18]:
cln_df = pd.merge(cln_df, ei_df, on='pt_root_id', how='left', suffixes=['_prf', '_ei'])
inh_cln_df = cln_df.query("cell_type=='inhibitory'")

In [20]:
inh_cln_df

,id_prf,valid_prf,pt_supervoxel_id_prf,pt_root_id,valid_id,status_dendrite,status_axon,pt_position_prf,id_ei,valid_ei,classification_system,cell_type,pt_supervoxel_id_ei,pt_position_ei
67,498,t,88893591900128821,864691136041323862,864691136041323862,clean,clean,"[175280, 194224, 22230]",267293.0,t,aibs_coarse_inhibitory,inhibitory,8.889359e+16,"[175280, 194224, 22230]"
68,499,t,91272797422269509,864691135771828043,864691135771828043,clean,clean,"[192656, 94768, 20588]",291125.0,t,aibs_coarse_inhibitory,inhibitory,9.127280e+16,"[192656, 94768, 20588]"
69,500,t,88676713231585599,864691135697462549,864691135697462549,clean,clean,"[173744, 151152, 22254]",260707.0,t,aibs_coarse_inhibitory,inhibitory,8.867671e+16,"[173744, 151152, 22254]"
70,501,t,90086012058979206,864691135754758610,864691135754758610,clean,clean,"[184256, 165088, 20542]",298796.0,t,aibs_coarse_inhibitory,inhibitory,9.008601e+16,"[184256, 165088, 20542]"
71,502,t,89239800355525240,864691135761944374,864691135761944374,clean,clean,"[178176, 152192, 20167]",260468.0,t,aibs_coarse_inhibitory,inhibitory,8.923980e+16,"[178176, 152192, 20167]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,594,t,88469661180018648,864691135584084344,864691135584084344,clean,clean,"[172240, 181184, 20446]",264792.0,t,aibs_coarse_inhibitory,inhibitory,8.846966e+16,"[172240, 181184, 20446]"
164,595,t,89033642126703313,864691135463706181,864691135463706181,clean,clean,"[176368, 188864, 21766]",264920.0,t,aibs_coarse_inhibitory,inhibitory,8.903364e+16,"[176368, 188864, 21766]"
165,596,t,88331122714745538,864691136483135020,864691136483135020,clean,clean,"[171248, 197696, 20273]",266850.0,t,aibs_coarse_inhibitory,inhibitory,8.833112e+16,"[171248, 197696, 20273]"
169,600,t,88686746275324078,864691135809729740,864691135809729740,clean,clean,"[174032, 225648, 22347]",271673.0,t,aibs_coarse_inhibitory,inhibitory,8.868675e+16,"[174032, 225648, 22347]"


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
    <p><b> Task 1.3: </b> Pick an inhibitory cell from the list, and find all its outputs.
        </div>

In [25]:
neuron_id = 864691135697462549
out_df = client.materialize.synapse_query(pre_ids=neuron_id)
out_df.shape

(9203, 10)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
    <p><b> Task 1.4:</b> Visualize this cell
        </div>

In [29]:
mm = trimesh_io.MeshMeta(cv_path ="precomputed://gs://iarpa_microns/minnie/minnie65/seg",
                         disk_cache_path='./em/meshes/',
                         cache_size=0)
mesh = mm.mesh(seg_id=neuron_id, lod=2)

ma = trimesh_vtk.mesh_actor(mesh, opacity=1.0)
trimesh_vtk.render_actors([ma])

setting up renderer
done setting up
actors added
camera set
render done
finalizing..


(vtkRenderingOpenGL2Python.vtkOpenGLRenderer)0x1a3346a4c8

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
    <p><b> Task 1.5: </b> Find the 10 largest synapses from this cell
        </div>


In [32]:
ten_big_df =out_df.sort_values('size', ascending=False).iloc[0:10]

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
    <p><b> Task 1.6:</b> Visualize the cell and these 10 synapses, with the synapse radius corresponding to the synapse size.
        </div>

In [42]:
ten_big_pos = np.vstack(ten_big_df.ctr_pt_position.values)*syn_table_resolution
pt_act = trimesh_vtk.point_cloud_actor(ten_big_pos, size=ten_big_df['size']*.05)

In [43]:
trimesh_vtk.render_actors([ma, pt_act])

setting up renderer
done setting up
actors added
camera set
render done
finalizing..


(vtkRenderingOpenGL2Python.vtkOpenGLRenderer)0x1a429de1c8